In [ ]:
# !pip install seaborn

In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
from pymongo import MongoClient
import seaborn as sns

In [ ]:
client = MongoClient('mongodb://localhost:27017')
collection = client.GFM.Data_12_27_20
dbData = pd.DataFrame(collection.find({"Status":"Active","Category":"Medical"}))
display(dbData.head())

In [ ]:
# dbData.drop(["_id","Position","Status","CurrencyCode","Duration","ErrorMessage","Organiser"], axis = 1, inplace = True)
len(dbData)

In [ ]:
#DataError : goal = 0 means Amount_Raised = 0
idx = (dbData.Goal == 0)
dbData.loc[idx] = dbData.loc[idx].rename(columns={'Amount_Raised':'Goal','Goal':'Amount_Raised'})
dbData["Progress"] = dbData.Amount_Raised/dbData.Goal
dbData.head()

In [ ]:
dbData.dropna(subset=['Goal'], inplace = True)

In [ ]:
display(dbData.Progress.hist())

In [ ]:
x = dbData[dbData.Progress > 10]
print(len(x))
x[["Url","Progress"]]

In [ ]:
data = pd.read_csv("data/GFM_dataRevised.csv", sep = "\t")
print(data.shape)
data.head()

Number of fundraiser's with amount_raised = goal


(Check if amount raised = 0)

In [ ]:
_ = data[data.Amount_Raised == data.Goal]
print(len(_))

In [ ]:
print(_.Url.iloc[7])
print(_.Url.iloc[0])

In [ ]:
headers = ["Url", "Category","Position", "Title", "Location", "Organiser","Amount_Raised", "Goal", "Duration", "Text", "Image"]

In [ ]:
listInvalidCampaign = []
lstScrapingError = []

def scrape_url(url, tid = "mainThread"):
    print("Scraping url %s", url)
        
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'lxml')
    
    temp_df = pd.DataFrame(columns = headers)
    
#     invalidCampaign = soup.find('div', attrs={'class': 'm-campaign-sidebar-notification m-well m-well--sand'})
    
#     if(invalidCampaign is not None):
#         text_container =  soup.find('meta', attrs={'name': 'description'})
#         print(text_container['content'])
#         listInvalidCampaign.append(row_index)
#         return "Invalid Campaign"
    
    try:
        # Description
        text_container =  soup.find('meta', attrs={'name': 'description'})
        try:
            all_text = text_container['content']
        except:
            all_text = np.nan

        if(all_text is np.nan )or(all_text == "We're sorry, but that campaign cannot be found. Please check the link URL and try again."):
            print(tid+' ('+str(row_index)+') -> '+all_text)
            listInvalidCampaign.append(row_index)
            temp_df = pd.DataFrame(columns = headers)
            return(temp_df)

        # Title
        try:
            title = soup.find("title").text
        except:
            title = np.nan

        # Organiser & Location
        try:
            t = soup.find_all("div",{"class":"p-campaign-members"})[0]
            organiser = t.find_all("div",{"class": "m-person-info-name"})[0].text.replace('\xa0','')
            location = ""
            if(len(t.find_all("div",{"class": "text-small"})) > 3):
                location = t.find_all("div",{"class": "text-small"})[2].text.replace('\xa0','')
            else:
                location = t.find_all("div",{"class": "text-small"})[1].text.replace('\xa0','')
        except:
            organiser = np.nan
            location = np.nan

        # Amount Raised and Goal
        try:
            t = soup.find("div",{"class":"p-campaign-sidebar"})
            amount = t.find("div",{"class": "o-campaign-sidebar-progress-meter m-progress-meter"})
            amountRaised = amount.text.split()[0]
            goal = amount.text.split()[-2]
        except:
            amountRaised = np.nan
            goal = np.nan

        # All Text
        soup.find('div', {'class': "p-campaign-description"})
        text = soup.find('div', {'class': "o-campaign-story mt-three-halves"}).text.replace('\xa0','').replace('\n','')
        text

        # Duration of Fundraising
        try:
            t = soup.find("div",{"class": "p-campaign-description"})
            startDate = ' '.join(t.find("li",{"class": "m-meta-list-item"}).text.split()[1::])
            startDate = (pd.to_datetime(startDate)).date()
            duration = (pd.datetime.now().date() - startDate).days
        except:
            duration = np.nan

        #Image
        try:
            img_container =  soup.find('meta', attrs={'property': 'og:image'})
            image_url = img_container['content']

            img_filename = "data/images/"+image_url.split("/")[-1]

            # Open the url image, set stream to True, this will return the stream content.
            r = requests.get(image_url, stream = True)

            # Check if the image was retrieved successfully
            if r.status_code == 200:
                # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
                r.raw.decode_content = True

                # Open a local file with wb ( write binary ) permission.
                with open(img_filename,'wb') as f:
                    shutil.copyfileobj(r.raw, f)
        except:
            img_filename = np.nan

    #     # FB likes and Shares    
    #     try:
    #         FB_shares_container = soup.find_all("strong", {"class":"js-share-count-text"})
    #         FB_shares = FB_shares_container[0].text.splitlines()
    #         FB_shares = FB_shares[1].replace(" ", "").replace("\xa0", "")
    #     except:
    #         FB_shares = np.nan

    #     try:
    #         hearts_container = soup.find_all("div", {"class":"campaign-sp campaign-sp--heart fave-num"})
    #         hearts = hearts_container[0].text
    #     except:
    #         hearts = np.nan

        temp_row = np.array([[url, category, position, title, location, organiser, amountRaised, goal, duration, text, img_filename]])
        temp_df = pd.DataFrame(temp_row, columns = headers)

        return(temp_df)
    
    except:
        lstScrapingError.append(row_index)
        return tid+' ('+str(row_index)+') -> '+"Scraping Error"

In [ ]:
scrape_url(_.Url.iloc[7])